In [15]:
import numpy as np
import xarray
import metpy
import os
from os import path as op
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from pyresample import image, geometry
import seaborn as sns
import dask
import string
from subprocess import Popen
sns.set(style="darkgrid")
%matplotlib inline

In [16]:
path = "G16_G17_url_list/"
dirs = os.listdir( path )

In [17]:
newPath = '../../../../../sharedData/scratch/npOut/'
for txtFile in dirs:
    txtFileName = txtFile[:-4]
    dirName = op.join(newPath, txtFileName)
    cmdList = [ 'mkdir -p '+dirName]
    print (cmdList)

    pMkdir = Popen(cmdList,shell = True)
    pMkdir.communicate()
    '''
    with open (txtFilePath) as f:
        readTxt = f.read()
        urlList = readTxt.split()
        for url in urlList:
            continue'''

['mkdir -p ../../../../../sharedData/scratch/npOut/G16_data_04_08-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G16_data_04_09-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G16_data_04_10-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G16_data_04_11-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G16_data_04_12-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G16_data_04_13-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G16_data_04_14-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G16_data_04_15-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G16_data_04_16-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G16_data_04_17-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G16_data_04_18-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G17_data_04_08-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G17_data_04_09-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G17_data_04_10-19']
['mkdi